In [1]:
from gensim.models import Word2Vec
from tqdm import tqdm
import re
import pickle
import jieba

re_apos = re.compile(r"(\w)'s\b")         # make 's a separate word
re_mw_punc = re.compile(r"(\w[’'])(\w)")  # other ' in a word creates 2 words
re_punc = re.compile("([\"().,;:/_?!—])") # add spaces around punctuation
re_mult_space = re.compile(r"  *")        # replace multiple spaces with just one

def simple_toks(sent):
    sent = re_apos.sub(r"\1 's", sent)
    sent = re_mw_punc.sub(r"\1 \2", sent)
    sent = re_punc.sub(r" \1 ", sent).replace('-', ' ')
    sent = re_mult_space.sub(' ', sent)
    return sent.lower().split()


class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for line in tqdm(open(self.dirname)):
            yield simple_toks(line)
            
class MyChineseSentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for line in tqdm(open(self.dirname)):
            yield jieba.cut(line)            

C:\Users\Cheng\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [9]:
path='data/training/'

fname=path+'train'
en_fname = fname+'.en'
zh_fname = fname+'.zh'

def train_save_wordvec_export(lan ='en'):
    output = 'input/'+lan
    if lan == 'en':
        corpus = MySentences(en_fname)
    else:
        corpus = MyChineseSentences(zh_fname)
    model = Word2Vec(corpus, min_count=10, sg=1,workers=2,iter=50)    
    print('model trained!')
    vocabulary = model.wv.vocab
    embeddings = []
    for word in vocabulary:
        embeddings.append(model[word])    
    embeddings = np.array(embeddings, dtype=np.float32)
    pickle.dump(vocabulary, open(output+'vocab.pkl', "wb"))
    np.save(output+'word2vec.npy',embeddings)
    print('word vector saved!')

In [10]:
train_save_wordvec_export(lan ='en')
train_save_wordvec_export(lan ='zh')

0it [00:00, ?it/s]

UnicodeDecodeError: 'gbk' codec can't decode byte 0x9f in position 453: illegal multibyte sequence